# Градиентный бустинг
## 1. Сравнение градиентного бустинга и решающего леса.

Сравним, как ведут себя бустинг и бэггинг с ростом числа базовых алгоритмов.

В случае бэггинга все базовые алгоритмы настраиваются на различные выборки из одного и того же распределения. При этом некоторые из них могут оказаться переобученными, однако усреднение позволяет ослабить этот эффект (объясняется тем, что для некоррелированных алгоритмов разброс композиции оказывается в $N$ раз меньше разброса отдельных алгоритмов, т.е. много деревьев с меньшей вероятностью настроятся на некоторый нетипичный объект по сравнению с одним деревом). Если $N$ достаточно велико, то последующие добавления новых алгоритмов уже не позволят улучшить качество модели.

В случае же бустинга каждый алгоритм настраивается на ошибки всех предыдущих, это позволяет на каждом шаге настраиваться на исходное распределение все точнее и точнее. Однако при достаточно большом $N$ это является источником переобучения, поскольку последующие добавления новых алгоритмов будут продолжать настраиваться на обучающую выборку, уменьшая ошибку на ней, при этом уменьшая обобщающую способность итоговой композиции.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

- сгенерируем точки, разбитые на две группы: внутри каждой группы точки лежат в окрестности некоторой прямой
- наша цель: понять, смогут ли бэггинг и градиентный бустинг понять эту зависимость (т.е. понять, что точки внутри каждйо группы лежат в окрестности некоторой прямой)

In [ ]:
X_train = np.linspace(0, 1, 100)
X_test = np.linspace(0, 1, 1000)

def target(x):
    return x > 0.5

Y_train = target(X_train) + np.random.randn(*X_train.shape) * 0.1

plt.scatter(X_train, Y_train, s=50)

Начнём с бэггинга, а именно, с решающего леса.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from tqdm import tqdm

reg = RandomForestRegressor(max_depth=2)
plt.figure(figsize=(20, 30))
sizes = [1, 2, 5, 20, 100, 500, 1000, 2000]
for i, s in tqdm(enumerate(sizes)):
    reg.n_estimators = s
    reg.fit(X_train.reshape(-1, 1), Y_train)
    plt.subplot(4, 2, i+1)
    plt.xlim([0, 1])
    plt.scatter(X_train, Y_train, s=30)
    plt.plot(X_test, reg.predict(X_test.reshape(-1, 1)), c='green', linewidth=4)
    plt.title('{} trees'.format(s))

Можно заметить, что с некоторого момента итоговая функция перестает меняться с ростом количества деревьев.

Теперь проделаем то же самое для градинентного бустинга.

In [ ]:
reg = GradientBoostingRegressor(max_depth=1, learning_rate=1)
plt.figure(figsize=(20, 30))
sizes = [1, 2, 5, 20, 100, 500, 1000, 2000]
for i, s in tqdm(enumerate(sizes)):
    reg.n_estimators = s
    reg.fit(X_train.reshape(-1, 1), Y_train)
    plt.subplot(4, 2, i+1)
    plt.xlim([0, 1])
    plt.scatter(X_train, Y_train, s=30)
    plt.plot(X_test, reg.predict(X_test.reshape(-1, 1)), c='green', linewidth=4)
    plt.title('{} trees'.format(s))

Градиентный бустинг довольно быстро построил истинную зависимость, после чего начал настраиваться уже на конкретные объекты обучающей выборки, из-за чего сильно переобучился.


Бороться с этой проблемой можно с помощью выбора очень простого базового алгоритма или
же искусственным снижением веса новых алгоритмов при помощи ***шага $\eta$***:
$$a_N(x) = \sum_{n=0}^N \eta \gamma_N b_n(x).$$

***Такая поправка замедляет обучение по сравнению с бэггингом, но зато позволяет получить менее переобученный алгоритм***. Тем не менее, важно понимать, что переобучение всё равно будет иметь место при обучении сколь угодно большого количества базовых алгоритмов для фиксированного $\eta$.

In [ ]:
reg = GradientBoostingRegressor(max_depth=1, learning_rate=0.1, warm_start=True)
plt.figure(figsize=(20, 30))
sizes = [1, 2, 5, 20, 100, 500, 1000, 2000]
for i, s in tqdm(enumerate(sizes)):
    reg.n_estimators = s
    reg.fit(X_train.reshape(-1, 1), Y_train)
    plt.subplot(4, 2, i+1)
    plt.xlim([0, 1])
    plt.scatter(X_train, Y_train, s=30)
    plt.plot(X_test, reg.predict(X_test.reshape(-1, 1)), c='green', linewidth=4)
    plt.title('{} trees'.format(s))

## 2. Решение задачи регрессии с помощью градиентного бустинга и решающего леса.

Будем решать задачу регрессии на примере датасета **Diabetes**.
По различным факторам будем предсказывать количественный эффект от лечения диабетиков спустя год.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

ds = datasets.load_diabetes()
print(ds.DESCR)
X = ds.data
Y = ds.target

## Задание.

* Разбейте данные на train и test. 
* Объявите модель бустинга

In [ ]:
X_train, X_test, Y_train, Y_test = #your code here

MAX_ESTIMATORS = 250

model = #your code here

err_train_gb = []
err_test_gb = []

Для каждого числа деревьев в промежутке (1, MAX_ESTIMATORS+1) обучим на train градиентный бустинг и запишем в созданные списки полученную ошибку: 1 - r2.

In [ ]:
for i in range(1, MAX_ESTIMATORS+1):
    model.n_estimators = i
    model.fit(X_train, Y_train)
    err_train_gb.append(1 - model.score(X_train, Y_train))
    err_test_gb.append(1 - model.score(X_test, Y_test))

Повторите процедуру для решающего леса.

In [ ]:
model = RandomForestRegressor(random_state=222)
err_train_bag = []
err_test_bag = []

for i in range(1, MAX_ESTIMATORS+1):
    #your code here

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(err_train_gb, label='GB')
plt.plot(err_train_bag, label='RF')
plt.legend()
plt.title('Train')
plt.subplot(1, 2, 2)
plt.plot(err_test_gb, label='GB')
plt.plot(err_test_bag, label='RF')
plt.legend()
plt.title('Test')

* Мы видим, что случайный лес достигает некоторой минимальной ошибки на обучении и на тесте и дальше не меняет своего качества.

* При этом ошибка градиентного бустинга всё время уменьшается на обучении (стремится к нулю), а на тесте с какого-то момента начинает расти, то есть начинается переобучение.

### Ответим на следующие вопросы:
* ***Как подобрать количество деревьев в RandomForest для достижения наилучшего качества и наименьших временных затрат?***
* ***Как подобрать количество деревьев в GradientBoosting для достижения наилучшего качества и не переобучиться?***

**Ответим на первый вопрос**: как подобрать количество деревьев в RandomForest для достижения наилучшего качества и наименьших временных затрат? 

In [ ]:
err_train_bag = np.array(err_train_bag)
err_test_bag = np.array(err_test_bag)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(err_train_bag, label='RF')
plt.plot([0, err_train_bag.shape[0]], [err_train_bag.min(), err_train_bag.min()], 'g--')

plt.legend()
plt.title('Train')
plt.subplot(1, 2, 2)
plt.plot(err_test_bag, label='RF')
plt.plot([0, err_test_bag.shape[0]], [err_test_bag.min(), err_test_bag.min()], 'g--')
plt.legend()
plt.title('Test')

Мы видим, что начиная примерно с 50-60 деревьев в лесе ошибка на train и на test перестает уменьшаться и колеблется около своего минимума. Поэтому ***для обучения леса в этой задаче достаточно 50-60 деревьев***.

**Ответим на второй вопрос**: как подобрать количество деревьев в GradientBoosting для достижения наилучшего качества и не переобучиться? 

Так как *ошибка на трейне в бустинге всегда уменьшается*, то ***для снижения переобучения найдем количество деревьев, требуемое для достижения минимальной ошибки на тесте***.

In [ ]:
err_train_gb = np.array(err_train_gb)
err_test_gb = np.array(err_test_gb)

plt.figure(figsize=(10, 4))
plt.plot(err_test_gb, label='GB')
plt.legend()
plt.title('Test')

plt.plot([0, err_test_gb.shape[0]], [err_test_gb.min(), err_test_gb.min()], 'g--')
plt.plot([err_test_gb.argmin()], [err_test_gb.min()], 'v')
plt.title('test error=%.3f, best_est=%d' % (err_test_gb.min(), err_test_gb.argmin()+1))

Мы видим, что минимальная ошибка достигается при $n=25$ деревьях.

Сравним ошибку решающего леса и ошибку бустинга на тесте.

In [ ]:
print('Random Forest:', err_test_bag.min(), 'n_trees:', err_test_bag.argmin()+1)
print('Gradient Boosting:', err_test_gb.min(), 'n_trees:', err_test_gb.argmin()+1)

#### Выводы:

* Мы видим, что в данной задаче случайный лес дает качество, немного лучшее, чем градиентный бустинг. В других задачах может получиться обратная ситуация.

* Случайный лес не переобучается, поэтому для него не обязательно искать оптимальное количество деревьев (полезно только найти минимальное необходимое число деревьев, чтобы снизить время обучения).

* Для градиентного бустинга ***необходимо*** находить оптимальное количество деревьев, чтобы избежать переобучения.

Посмотрим, можно ли улучшить качество градиентного бустинга за счёт подбора гиперпараметров.

Без подбора параметров с оптимальным числом деревьев градиентный бустинг дает следующее качество:

In [ ]:
from sklearn.model_selection import cross_val_score

model = GradientBoostingRegressor(n_estimators=25, random_state=111)

cross_val_score(model, X, Y, cv=3, scoring='r2').mean()

Подберем основные параметры (n_estimators, max_features, max_depth) по сетке. Так как поиск по сетке занимает длительное время, число деревьев будем искать в маленьком диапазоне и с шагом 5.

![Поиск по сетке (GridSearch)](gridsearch.jpg)

In [ ]:
from sklearn.model_selection import GridSearchCV

%time
gs = GridSearchCV(GradientBoostingRegressor(random_state=111),
                  param_grid={'n_estimators': range(10,50,5),
                             'max_features': range(1,X.shape[1]+1), 
                             'max_depth': range(2,20)},
                  cv=3,
                  scoring='r2', verbose=1, n_jobs=-1)
gs.fit(X, Y)

In [ ]:
gs.best_score_, gs.best_params_

Устроим более полный поиск параметров по сетке: будем также искать learning_rate, min_samples_leaf, min_samples_split, subsample.

In [ ]:
%time
gs = GridSearchCV(GradientBoostingRegressor(random_state=111),
                  param_grid={'n_estimators': range(10,500),
                             'max_features': range(1,X.shape[1]+1), 
                             'max_depth': range(2,10),
                             'learning_rate': np.arange(0.1,1.1,0.1),
                             'min_samples_leaf': range(1,10),
                             'min_samples_split': np.arange(0.1,1.1,0.1),
                             'subsample': np.arange(0.1,1.1,0.1)},
                  cv=3,
                  scoring='r2', verbose=1, n_jobs=-1)
gs.fit(X, Y)

Мы видим, что поиск по большому числу параметров занимает очень много времени. Поэтому используем другой вариант подбора параметров, основанный на байесовских методах - **hyperopt**.

Для того, чтобы уменьшить число итераций до нахождения хорошей конфигурации, придуманы адаптивные байесовские методы. Они выбирают следующую точку для проверки, учитывая результаты на уже проверенных точках. Идея состоит в том, чтобы на каждом шаге найти компромисс между (а) исследованием регионов рядом с самыми удачными точками среди найденных и (б) исследованием регионов с большой неопределенностью, где могут находиться еще более удачные точки. Это часто называют дилеммой explore-exploit или «learning vs earning». Таким образом, в ситуациях, когда проверка каждой новой точки стоит дорого (в машинном обучении проверка = обучение + валидация), можно приблизиться к глобальному оптимуму за гораздо меньшее число шагов (подробнее см. https://habr.com/ru/company/dca/blog/272697/).

![Hyperopt](tpesearch.png)

In [ ]:
from hpsklearn import HyperoptEstimator, gradient_boosting_regression
from hyperopt import tpe

estim = HyperoptEstimator(regressor=gradient_boosting_regression('reg'),  
                            algo=tpe.suggest, max_evals=100)

estim.fit(X_train, Y_train)

print(estim.score(X_test, Y_test))
print(estim.best_model())

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

model = Pipeline([('Scaler', estim.best_model()['preprocs'][0]),
                 ('RF', estim.best_model()['learner'])])
#model = estim.best_model()['learner']

cross_val_score(model, X, Y, cv=3, scoring='r2').mean()

Используя hyperopt, можно подбирать параметры у любых других моделей.

# Домашнее задание.
Поработайте с датасетом для классификации https://archive.ics.uci.edu/ml/datasets/Alcohol+QCM+Sensor+Dataset#

Загрузим все таблички из датасета (QCM.csv) и объединим их в одну таблицу. 

Последние 5 колонок содержат в себе целевую переменную. Создадим одну колонку с целевой переменной y следующим образом: если 1-Octanol = 1, то y = 0, если 1-Propanol = 1, то y=1, если 2-Butanol = 1, то y=2 и т.д.

Создадим матрицу объект-признак X, содержащую все признаки (кроме последних 5 целевых колонок). 

1) Посчитайте качество DecisionTreeClassifier, RandomForestClassifier и GradientBoostingClassifier на кросс-валидации.

2*) Попробуйте уменьшить число признаков с помощью какого-либо метода отбора признаков (http://scikit-learn.org/stable/modules/feature_selection.html). Добейтесь увеличения качества на кросс-валидации.

3) Используйте gridsearch или hyperopt для подбора оптимальных параметров у леса и у бустинга. Какого наилучшего качества на кросс-валидации удалось достичь?  

In [ ]:
import pandas as pd

data1 = pd.read_csv("QCM3.csv",delimiter=';')
data2 = pd.read_csv("QCM6.csv",delimiter=';')
data3 = pd.read_csv("QCM7.csv",delimiter=';')
data4 = pd.read_csv("QCM10.csv",delimiter=';')
data5 = pd.read_csv("QCM12.csv",delimiter=';')

#your code here
df = pd.concat([data1, data2, data3, data4, data5])
df.tail()

In [ ]:
def get_target(x):
    if x['1-Octanol'] == 1:
        return 0
    if x['1-Propanol'] == 1:
        return 1
    if x['2-Butanol'] == 1:
        return 2
    if x['2-propanol'] == 1:
        return 3
    if x['1-isobutanol'] == 1:
        return 4
    return -1
    
df['Target'] = df.apply(get_target, axis=1)

df.drop(['1-Octanol','1-Propanol','2-Butanol','2-propanol','1-isobutanol'],\
        axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
X = df.drop('Target',axis=1)
y = df['Target']

In [ ]:
#your code here

dt = DecisionTreeClassifier()

print(cross_val_score(dt, X, y, cv=5, scoring='accuracy').mean())

from sklearn.ensemble import RandomForestClassifier

for n in range(11, 21):
    rf = RandomForestClassifier(n_estimators = n, max_depth=4)
    
    print('n = ',n)
    print(cross_val_score(rf, X, y, cv=5, scoring='accuracy').mean())